In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader


In [3]:
# Define the LeNet architecture
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=2)  # Input: 1x28x28, Output: 6x28x28
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5, stride=1)  # Input: 6x28x28, Output: 16x24x24
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  # Fully connected layer
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)  # Output layer for 10 classes

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.max_pool2d(x, kernel_size=2, stride=2)  # Output: 6x14x14
        x = torch.relu(self.conv2(x))
        x = torch.max_pool2d(x, kernel_size=2, stride=2)  # Output: 16x5x5
        x = x.view(-1, 16 * 5 * 5)  # Flatten
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)  # Output logits
        return x



In [4]:
# Data preparation
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Normalize to mean 0, std 1
])

train_dataset = datasets.MNIST(root="./data", train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root="./data", train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)



In [5]:
# Model, loss function, optimizer
model = LeNet()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)



In [6]:
# Training loop
def train(model, device, train_loader, optimizer, criterion, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)

        # Forward pass
        output = model(data)
        loss = criterion(output, target)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch_idx % 100 == 0:
            print(f"Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}")



In [7]:
# Testing loop
def test(model, device, test_loader, criterion):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)

            # Sum up batch loss
            test_loss += criterion(output, target).item()

            # Get the index of the max log-probability
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print(f"\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({100. * correct / len(test_loader.dataset):.0f}%)\n")


In [8]:
# Main training/testing process
num_epochs = 10
for epoch in range(1, num_epochs + 1):
    train(model, device, train_loader, optimizer, criterion, epoch)
    test(model, device, test_loader, criterion)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.303926
Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.221195
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.200158
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.073620
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.226836
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.156287
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.159469
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.100627
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.216015
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.158843

Test set: Average loss: 0.0001, Accuracy: 9736/10000 (97%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.137380
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.196022
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.104287
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.054255
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.075574
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.109670
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.024280
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.054277
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.056697
T